In [69]:
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [14]:
#the code assumes you have equal number of instances of each class to make the implementation simpler
num_i = 130 #number of instances per class
num_c = 4 #number of classes
beach = []
city = []
forest = []
mountains = []

for i in range(num_i):
    beach.append(mpimg.imread('Dataset/Beach/image'+str(i)+'.jpg'))
    city.append(mpimg.imread('Dataset/City/image'+str(i)+'.jpg'))
    forest.append(mpimg.imread('Dataset/Forest/image'+str(i)+'.jpg'))
    mountains.append(mpimg.imread('Dataset/Mountains/image'+str(i)+'.jpg'))
    
beach = np.array(beach)
city = np.array(city)
forest = np.array(forest)
mountains = np.array(mountains)

In [66]:
#we have just imported .jpg images as numpy arrays
print(city.shape, '---->', 'number of instaces, height, width, number of channels')
num_i, height, width, channels = city.shape

(130, 299, 299, 3) ----> number of instaces, height, width, number of channels


In [54]:
dataset = np.vstack((beach,city,forest,mountains)).reshape(num_i*num_c,-1) #unrolling each image into a long vector
dataset = dataset/255 #normalizing
labels = np.zeros(num_c*num_i)

for i in range(num_c):
    labels[i*num_i:(i+1)*num_i] = i

In [55]:
dataset.shape

(520, 268203)

In [97]:
labels = labels.astype(int)
labels.shape

(520,)

In [98]:
X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.1, shuffle=True, stratify=labels)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.11, shuffle=True, stratify=y_train)

In [99]:
print('X_train:', X_train.shape, '----> 90% of data')
print('X_val:', X_val.shape, '----> 10% of data')
print('X_test:', X_test.shape, '----> 10% of data')

X_train: (416, 268203) ----> 90% of data
X_val: (52, 268203) ----> 10% of data
X_test: (52, 268203) ----> 10% of data


In [100]:
np.unique(y_train, return_counts=True) #checking if the split is stratified

(array([0, 1, 2, 3]), array([104, 104, 104, 104], dtype=int64))

In [152]:
learning_rate = 0.001
reg_constant = 0.08
num_epochs = 100

In [153]:
#implementing logistic regression using TF
tf.reset_default_graph()

X = tf.placeholder(tf.float64, shape=(None, height*width*channels), name='X')
y = tf.placeholder(tf.int64, shape=(None), name='y')

with tf.name_scope('multiplication'):
    logits = fully_connected(X, num_c, activation_fn=None, scope="outputs")
    
with tf.name_scope('loss'):
    tr_var = tf.trainable_variables()
    reg_loss = tf.reduce_sum([tf.nn.l2_loss(v) for v in tr_var if 'bias' not in v.name]) * reg_constant
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)) + reg_loss
    
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope('predict'):
    softmax = tf.nn.softmax(logits)
    predictions = tf.argmax(softmax, axis=1)
    
with tf.name_scope('eval'):
    good_pred = tf.cast(tf.equal(predictions, y), tf.float64)
    accuracy = tf.reduce_mean(good_pred)
    
init = tf.global_variables_initializer()

In [154]:
with tf.Session() as sess:
    
    sess.run(init)
    
    for epoch in range(num_epochs):
        sess.run(training_op, feed_dict={X: X_train, y: y_train})
        if epoch % 10 == 0:
            tr_loss = loss.eval(feed_dict={X: X_train, y: y_train})
            training_accuracy = accuracy.eval(feed_dict={X: X_train, y: y_train})
            validation_accuracy = accuracy.eval(feed_dict={X: X_val, y: y_val})
            print('Loss:', tr_loss)
            print('Training accuracy:', training_accuracy)
            print('Validation accuracy:', validation_accuracy)
    
    print('-----------------------')
    test_accuracy = accuracy.eval(feed_dict={X: X_test, y: y_test})
    print('Test accuracy:', test_accuracy)

Loss: 152.222625951
Training accuracy: 0.25
Validation accuracy: 0.25
Loss: 31.3866564797
Training accuracy: 0.487980769231
Validation accuracy: 0.519230769231
Loss: 22.367673214
Training accuracy: 0.526442307692
Validation accuracy: 0.5
Loss: 19.2944122588
Training accuracy: 0.641826923077
Validation accuracy: 0.673076923077
Loss: 10.4670918894
Training accuracy: 0.711538461538
Validation accuracy: 0.615384615385
Loss: 7.58961251442
Training accuracy: 0.75
Validation accuracy: 0.711538461538
Loss: 3.79243498405
Training accuracy: 0.817307692308
Validation accuracy: 0.653846153846
Loss: 1.99876342252
Training accuracy: 0.855769230769
Validation accuracy: 0.692307692308
Loss: 1.11034908214
Training accuracy: 0.894230769231
Validation accuracy: 0.692307692308
Loss: 0.779934392771
Training accuracy: 0.949519230769
Validation accuracy: 0.730769230769
-----------------------
Test accuracy: 0.788461538462
